## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-10-14-51-09 +0000,bbc,"Don't let Trump interfere with UK democracy, D...",https://www.bbc.com/news/articles/cvg17q98zp4o...
1,2025-12-10-14-50-53 +0000,bbc,US sanctions network it says recruits Colombia...,https://www.bbc.com/news/articles/c93nxqw00xxo...
2,2025-12-10-14-48-42 +0000,nypost,Marco Rubio instructs diplomats to use Times N...,https://nypost.com/2025/12/10/us-news/marco-ru...
3,2025-12-10-14-48-10 +0000,cbc,Bank of Canada holds key interest rate at 2.25%,https://www.cbc.ca/news/business/bank-of-canad...
4,2025-12-10-14-45-21 +0000,nyt,Venezuela’s Nobel Peace Prize Winner Says She ...,https://www.nytimes.com/2025/12/10/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2355/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
2,trump,42
23,new,15
467,media,14
228,out,14
43,will,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
61,2025-12-10-11-56-00 +0000,wsj,The next Fed chair will have to walk the tight...,https://www.wsj.com/politics/policy/can-the-ne...,118
126,2025-12-10-09-00-00 +0000,cbc,Is Trump’s stark new security strategy the end...,https://www.cbc.ca/news/world/trump-world-orde...,95
189,2025-12-10-01-42-00 +0000,wsj,Eileen Higgins became the first Democrat to wi...,https://www.wsj.com/politics/elections/democra...,92
183,2025-12-10-02-08-00 +0000,wsj,President Trump touted the economy as he embar...,https://www.wsj.com/politics/policy/trump-tout...,88
203,2025-12-10-00-37-01 +0000,nypost,Trump kicks off affordability messaging tour a...,https://nypost.com/2025/12/09/us-news/trump-ki...,81


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
61,118,2025-12-10-11-56-00 +0000,wsj,The next Fed chair will have to walk the tight...,https://www.wsj.com/politics/policy/can-the-ne...
152,71,2025-12-10-05-05-52 +0000,bbc,Kids locked out of social media apps after Aus...,https://www.bbc.com/news/articles/cgl6gkd7pz6o...
6,61,2025-12-10-14-42-00 +0000,wsj,"New York City Comptroller Brad Lander, a progr...",https://www.wsj.com/politics/policy/nyc-comptr...
80,53,2025-12-10-11-03-45 +0000,nypost,DOJ civil rights chief Harmeet Dhillon reveals...,https://nypost.com/2025/12/10/us-news/harmeet-...
172,52,2025-12-10-03-16-48 +0000,cbc,'Venezuela will breathe again': María Corina M...,https://www.cbc.ca/news/world/nobel-peace-priz...
203,39,2025-12-10-00-37-01 +0000,nypost,Trump kicks off affordability messaging tour a...,https://nypost.com/2025/12/09/us-news/trump-ki...
241,39,2025-12-09-22-33-02 +0000,nypost,"Netflix, Paramount shares sink as Wall Street ...",https://nypost.com/2025/12/09/media/netflix-pa...
295,38,2025-12-09-19-12-00 +0000,wsj,"Bruce Blakeman, a Long Island Republican who t...",https://www.wsj.com/politics/elections/long-is...
292,35,2025-12-09-19-30-00 +0000,wsj,Nvidia AI Chips to Undergo Unusual U.S. Securi...,https://www.wsj.com/tech/ai/nvidia-ai-chips-to...
237,32,2025-12-09-22-50-28 +0000,latimes,Supreme Court poised to strike down Watergate-...,https://www.latimes.com/politics/story/2025-12...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
